#### Project 6

In this project you will simulate secure communication between two parties. There are two roles: A _sender_ and a _receiver_. The sender wants to do the following things:

1. Digitally sign a message (file) using [RSA PSS](https://en.wikipedia.org/wiki/Probabilistic_signature_scheme) to produce a file called `msg.sig`.
1. Encrypt the message using AES with a 128 bit key $K$ to produce a file called `msg.crypt`.
1. Encrypt $K$ using RSA with the receiver's public key and save to a file called `symkey.crypt`.
1. "Transmit" `msg.sig`,`msg.crypt`,`sym.key` (you don't actually have to do anything for this step).

The receiver wants to do the following:

1. Use her private key to decrypt `symkey.crypt` to produce `symkey`.
1. Use `symkey` to decrypt `msg.crypt` to produce `msg`.
1. Use the sender's public key to verify that `msg.sig` is a valid signature for `msg`.

You should produce two applications that work from the command line: **send** and **receive**.

The **send** program should take three command line inputs, all of which are filenames:

1. The name of the message file.
2. The name of the sender's private key file (for signing).
3. The name of the receiver's private key file (for encrypting the symmetric key).

There is code below that you can adapt for producing RSA keys (public and private) as well as a cryptographically secure 128 bit AES key.

Please name you files as follows. **Please actually use these names:) (many people don't but you _must_).**

1. Call the AES key file `symkey`.
1. Call the sender's private key `send_priv.der` (we will use keys in the DER format).
1. Call the sender's public key `send_pub.der`.
1. Call the receiver's private key `rec_priv.der`.
1. Call the receiver's public key `rec_pub.der`.

The **receive** program should take three command line inputs, all of which are file

In [1]:
### Warning:  Kernel must be Python 3 (System Wide)
### Select this from Kernel pulldown menu in Cocalc


In [2]:
!openssl genrsa -out private.pem 2048
!openssl rsa -in private.pem -out private.der -outform DER
!openssl rsa -in private.pem -outform DER -pubout -out public.der
!openssl rand 16 > aes.key

Generating RSA private key, 2048 bit long modulus (2 primes)
.................................................+++++
................................................+++++
e is 65537 (0x010001)
writing RSA key
writing RSA key


In [3]:

### Adapted from
### https://support.ssl.com/Knowledgebase/Article/View/19/0/der-vs-crt-vs-cer-vs-pem-certificates-and-how-to-convert-them
### and
### https://pycryptodome.readthedocs.io/en/latest/src/signature/pkcs1_pss.html



In [9]:

from Crypto.Signature import pss
from Crypto.Hash import SHA256
from Crypto.PublicKey import RSA
from Crypto import Random

message = open('msg.txt','r').read()


def sign_message(message,privkeyfile_der):
    key = RSA.import_key(open(privkeyfile_der,'rb').read())
    h = SHA256.new(message.encode())
    signature = pss.new(key).sign(h)
    return signature
    
sig = sign_message(message,'send_priv.der')    
sig

b'D\xb7\x8f\xc3C6\xa2a"l\xa0\xac@\x93\xf5\x89\x82\x08\x13\x9e\xea@\\\xab\xa7u\xca\xd7WmC<5P\x02\xd4F\x99\xe7e\x80\x8b\xc3i8\xa7\x14zE\xefz\xf8\xab\xe0 4\xe7\xd1q\x0e\x9d\xbb\xf5*\xf2\x93^q\x081\xb5z\xaf\x80\xda\xcf\xf6p;\x05@\x1ek\x7f\xdb\xc8\x12+5\xac\x95\xf9\x06d\x03\x01\xa47\x85\xf9\xf8H\x92\xe4u\xbeQ\x17\xac\xf8\x8f\xe9\xc1\x82\x8a\xbe\x1dJ3+M\x00N\n-]\xcbz\xb9\xa5\xc1\xbd\xaa\xf2\x84\x9a\xb3\xe5\x8c\xa0\xdd\xbe_\xe5W\xfa\xbd6V\xd9\xf5)\xcc\x97\x98g.\xb6\xd6%C\x02\xb8\x11~+\xd5_3\xc0\x99\x1e\x97>\xd2A\xaf\x13VKe\xc3P\xa3\x14\xf9\x0c\xe4\x95\x94$,\xdf0\xdc\xc6\x9e\xc1E\\\xcb\xe6\xcb\xdd\xd61\x8e\xd3\xe3\xdf\xcc\xa2\x87\xf5Z\x1e3\x15 \x84\xf0\xee\xb3d[G/"\x15{\x87\x08\xf0K7\xd6\rg\xc8\xf5&\x0e\x88K\xbb\xad"~\x01\xe6\xe9#\xd6\x9d\x89@'

In [10]:


def verify_sig(message,signature,pubkeyfile_der):
    key = RSA.import_key(open(pubkeyfile_der,'rb').read())
    h = SHA256.new(message.encode())
    verifier = pss.new(key)
    try:
        verifier.verify(h, signature)
        print( "The signature is authentic.")
    except (ValueError, TypeError):
        print( "The signature is not authentic.")

verify_sig(message,sig,"send_pub.der")

The signature is authentic.


In [17]:
from Crypto.Cipher import PKCS1_OAEP
from Crypto.PublicKey import RSA
from Crypto.Hash import SHA

message = open("symkey",'rb').read()

def RSAencrypt(message,pubkeyfile_der):
    key = RSA.importKey(open(pubkeyfile_der,'rb').read())
    cipher = PKCS1_OAEP.new(key)  ### Padding Scheme
    ciphertext = cipher.encrypt(message)
    return ciphertext
ciphertext = RSAencrypt(message,"rec_pub.der")
ciphertext

b'\x05g\xffy\xee/\x8f\xa8\x99\x88w\xfc\xcalp\x96\x04T\xa0\xc8\x99$)\xeb\t\xcc\x12\x183r\x97e\x14\xa0+\xae\xb6\x9b\x844w\xb0a\x0fH@\xb2d\x88U\xec\x05\xd2\xe7k\xa7.\xeeh\xa6\x9d\xb3\r_\xe5c\xc7\x02Y\x17\x9f\x15\xc4\xf9\xc8\xa3\xeaaS\x89\xd1\x01\xa2^\xadR\xf6$a\xae\xb8O\x83\rk=\xe1\x9b\xc1\xf2X\x02m\x98\x1e\xbaJ \x1dR\x1c\xf7$\xa0\xae\xb3\xeeMj\x8a\xf2\xaf\x0b\xccIEh\\\xaa\x96\xec\x8e\x14\xc4\xd2A\x93\xcfCp\xdc\xae\xd6\x8e\xcbNN4\xe4\x11d\x01\x9dx\x0e\xf2\t\x86\t\x0c\xdf\x19bSdO\x9d\xc6Z\xd7\xf9\x05\x04\xbf\xcd\xe5\xd2\xbf\x8d\xaaI\xf9\x8b\x7fWm{O\xd8l\x836\xedO\xb4u]FU\x80\xbe\xccc\x9d\x968\xf1\x0bP\xec\xfb\xaf\xe2\xab:/\xc5\xd7d\\\xc5\x88\xeb\xf18\x160\xb2\x11~\xcd\xfe\xfc\x94\xd3\xc8n\xcb\xa6\x11N\x01A<$\xce\xc2\x0f\x11U\xa2\\\xffM\x92\xca'

In [20]:
from Crypto.Hash import SHA
from Crypto import Random
from Crypto.PublicKey import RSA
from Crypto.Cipher import PKCS1_OAEP

def RSAdecrypt(ciphertext,privatekey_der):
    key = RSA.importKey(open(privatekey_der,'rb').read())
    #dsize = SHA.digest_size
    #sentinel = Random.new().read(15+dsize)      # Let's assume that average data length is 15
    cipher = PKCS1_OAEP.new(key)
    message = cipher.decrypt(ciphertext)
    return message

ciphertext = open("symkey.crypt","rb").read()
print(RSAdecrypt(ciphertext,"rec_priv.der"))

ValueError: Ciphertext too large